<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/dagngyen/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<style>
    h1 {
        padding: 8px 8px;
        background-image: linear-gradient(135deg, #c9f3ff, rgb(131, 218, 255));
        font-weight: 700;
        position: static;
        text-align: center;
        color: #006098;
        font-size: 36px;
    }
    h2 {
        font-weight: 700;
        text-align: center;
        font-style: italic;
        font-size: 24px;
    }
</style>

<div><h1>XÂY DỰNG MODEL</h1></div>
<div><h2>RAPIDS cuML SVR model</h2></div>

# Khai báo thư viện cần thiết

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



# Đọc dữ liệu

In [2]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [3]:
# train = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/train.csv')
# train.sample(5)

In [4]:
# test = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/test.csv')
# test

In [5]:
train = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/train.csv')
train.sample(5)

,essay_id,full_text,score
10396,998557d,Started with the founding fathers. The Elector...,3
4424,41bd378,The challenge of exploring venus is very risky...,2
13455,c591447,I think theat limiting car usage in some citie...,3
3111,2e3312c,Have you evr wanted to help take care of anima...,3
16401,f1c81a1,"""The callenge of Exploring Venus,"" the author ...",2


In [6]:
test = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/test.csv')
test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


---

In [4]:
train = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/train.csv')
test = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/test.csv')

In [5]:
train.sample(5)

,essay_id,full_text,score
8727,814c6b5,I think that limiting car usage is great for t...,4
15412,e2f5624,Classrooms in this time period are filled with...,6
16544,f3fd32c,I don't think driverless cars would be a good ...,3
7525,70ae3fd,The Conspiracy that aliens created the Face on...,3
10730,9e599a1,"Many people have heard of the ""Face"" of Mars. ...",3


In [6]:
test

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [1]:
import re
! pip install nltk
import nltk 
import warnings
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
! pip install spacy
from spacy import load
from nltk.tokenize import word_tokenize
from urllib.parse import urlparse
warnings.filterwarnings('ignore')

nltk.download("omw-1.4") # Open Multilingual WordNet
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("wordnet2022")
nltk.download("punkt")

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


[nltk_data] Downloading package omw-1.4 to /home/dagngyen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dagngyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dagngyen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to
[nltk_data]     /home/dagngyen/nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!
[nltk_data] Downloading package punkt to /home/dagngyen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
lemmatier = WordNetLemmatizer()
stop_words = stopwords.words("english")

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
def txt_process(text):
    try:
        # removing brackets
        text = re.sub("[][)(]"," ",text)

        # urls removing from text
        text = [word for word in text.split() if not urlparse(word).scheme]
        text = ' '.join(text)

        # removing escap characters
        text = re.sub(r'\@\w+','',text)

        # removing html tags from text
        text = re.sub(re.compile("<.*?>"),'',text)
        
        # remove words and numbers together form
        text = re.sub(r'\w*[0-9]\w*','',text)

        # lemmatization text
        text = [lemmatier.lemmatize(word) for word in text.split()]
        text = " ".join(text)

        # lower the text
        text = text.lower()

        # getting characters and numbers only
        text = re.sub('[^a-z0-9]',' ',text)

        # stripping all words from sentences
        text = [word.strip() for word in text.split()]
        text = ' '.join(text)
        
        # removing stopwords from text
        tokens = word_tokenize(text)
        text = []
        for word in tokens:
            if word.strip() in stop_words:
                pass
            else:
                text.append(word)
        text = ' '.join(text)

        return text
    except Exception as e:
        print("text ",text)
        print("Error ",e)

In [9]:
train['cleaned'] = train['full_text'].apply(lambda text:txt_process(text))

In [10]:
train.head()

,essay_id,full_text,score,cleaned
0,000d118,Many people have car where they live. The thin...,3,many people car live thing know use car alot t...
1,000fe60,I am a scientist at NASA that is discussing th...,3,scientist nasa discussing face mars explaining...
2,001ab80,People always wish they had the same technolog...,4,people always wish technology seen movies best...
3,001bdc0,"We all heard about Venus, the planet without a...",4,heard venus planet without almost oxygen earth...
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,dear state senator letter argue favor keeping ...


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['cleaned'])

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, train['score'], test_size=0.2, random_state=42)

In [38]:
from sklearn.svm import SVR

svr = SVR(kernel='linear')
svr.fit(X_train, y_train)

SVR(kernel='linear')

In [39]:
y_pred = svr.predict(X_test)

In [26]:
y_pred.min(), y_pred.max(), y_test.min(), y_test.max()

(0.132516116122247, 6.002278066198443, 1, 6)

In [40]:
y_pred_int = np.round(y_pred)
y_pred_int[y_pred_int < 1] = 1
y_pred_int[y_pred_int > 6] = 6

In [33]:
from sklearn.metrics import r2_score, mean_squared_error
mse = mean_squared_error(y_test, y_pred_int)
r2 = r2_score(y_test, y_pred_int)
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

Mean Squared Error: 0.5950317735413057
R2 Score: 0.4606845370703948


In [41]:
# F1 Score
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred_int, average='macro')
print(f'F1 Score: {f1}')

# Accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred_int)
print(f'Accuracy: {acc}')

F1 Score: 0.39135499867744955
Accuracy: 0.514153668399769


In [35]:
# XGBRegressor
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
y_pred_int = np.round(y_pred)
y_pred_int[y_pred_int < 1] = 1
y_pred_int[y_pred_int > 6] = 6
mse = mean_squared_error(y_test, y_pred_int)
r2 = r2_score(y_test, y_pred_int)
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

Mean Squared Error: 0.5465049104563836
R2 Score: 0.5046675456976636


In [37]:
# F1 Score
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred_int, average='macro')
print(f'F1 Score: {f1}')

# Accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred_int)
print(f'Accuracy: {acc}')

F1 Score: 0.4356200160861456
Accuracy: 0.5548815713460428


In [ ]:
# ! python -m spacy download en_core_web_sm

---

In [44]:
train = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/train.csv')
test = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/test.csv')

In [45]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
def dataPreprocessing(x):
    x = x.lower()
    x = removeHTML(x)
    x = re.sub("@\w+", '',x)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub("http\w+", '',x)
    x = re.sub(r"\s+", " ", x)
#     x = expandContractions(x)
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = x.strip()
    return x

Feature Engineering

In [49]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

with open('./embedding/words.txt', 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)
    
def count_spelling_errors(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_.lower() for token in doc]
    spelling_errors = sum(1 for token in lemmatized_tokens if token not in english_vocab)
    return spelling_errors

In [50]:
import string
def remove_punctuation(text):
    """
    Remove all punctuation from the input text.
    
    Args:
    - text (str): The input text.
    
    Returns:
    - str: The text with punctuation removed.
    """

    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [52]:
def Paragraph_Preprocess(tmp):

    tmp = tmp.explode('paragraph')
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(remove_punctuation).alias('paragraph_no_pinctuation'))
    tmp = tmp.with_columns(pl.col('paragraph_no_pinctuation').map_elements(count_spelling_errors).alias("paragraph_error_num"))
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp

# feature_eng
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
paragraph_fea2 = ['paragraph_error_num'] + paragraph_fea
def Paragraph_Eng(train_tmp):
    num_list = [0, 50,75,100,125,150,175,200,250,300,350,400,500,600]
    num_list2 = [0, 50,75,100,125,150,175,200,250,300,350,400,500,600,700]
    aggs = [
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_>{i}_cnt") for i in [0, 50,75,100,125,150,175,200,250,300,350,400,500,600,700] ], 
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_<{i}_cnt") for i in [25,49]], 
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea2],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea2],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea2],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in paragraph_fea2],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea2],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea2],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in paragraph_fea2],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in paragraph_fea2],
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in paragraph_fea2],
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df
tmp = Paragraph_Preprocess(train)
train_feats = Paragraph_Eng(tmp)
train_feats['score'] = train['score']

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

KeyError: 'paragraph'

In [ ]:
def Sentence_Preprocess(tmp):
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=".").alias("sentence"))
    tmp = tmp.explode('sentence')
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))    
    return tmp

# feature_eng
sentence_fea = ['sentence_len','sentence_word_cnt']
def Sentence_Eng(train_tmp):
    aggs = [
        *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_>{i}_cnt") for i in [0,15,50,100,150,200,250,300] ], 
        *[pl.col('sentence').filter(pl.col('sentence_len') <= i).count().alias(f"sentence_<{i}_cnt") for i in [15,50] ], 
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in sentence_fea],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in sentence_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in sentence_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in sentence_fea],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in sentence_fea],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in sentence_fea],
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in sentence_fea],
    
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Sentence_Preprocess(trainpl)
train_feats = train_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

# tmp = Sentence_Preprocess(trainpl)
# train_feats = Sentence_Eng(tmp)
# train_feats['score'] = trainpl['score']

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/tmp/ipykernel_24/1380819925.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
/tmp/ipykernel_24/1380819925.py:5: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))


Features Number:  81


,essay_id,paragraph_>0_cnt,paragraph_>50_cnt,paragraph_>75_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,...,sentence_len_first,sentence_word_cnt_first,sentence_len_last,sentence_word_cnt_last,sentence_len_kurtosis,sentence_word_cnt_kurtosis,sentence_len_q1,sentence_word_cnt_q1,sentence_len_q3,sentence_word_cnt_q3
0,000d118,1,1,1,1,1,1,1,1,1,...,36,7,0,1,1.438632,2.175806,109.0,19.0,225.0,37.0
1,000fe60,5,5,5,5,5,5,5,4,3,...,62,13,0,1,0.917062,0.505776,51.0,12.0,124.0,25.0
2,001ab80,4,4,4,4,4,4,4,4,4,...,144,27,0,1,-0.004393,0.270079,86.0,17.0,151.0,29.0


In [ ]:
# word feature
def Word_Preprocess(tmp):
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=" ").alias("word"))
    tmp = tmp.explode('word')
    tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))
    tmp = tmp.filter(pl.col('word_len')!=0)    
    return tmp

# feature_eng
def Word_Eng(train_tmp):
    aggs = [
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_{i+1}_cnt") for i in range(15) ], 
        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        pl.col('word_len').quantile(0.25).alias(f"word_len_q1"),
        pl.col('word_len').quantile(0.50).alias(f"word_len_q2"),
        pl.col('word_len').quantile(0.75).alias(f"word_len_q3"),
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Word_Preprocess(trainpl)
train_feats = train_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/tmp/ipykernel_24/2907905457.py:5: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))


Features Number:  102


,essay_id,paragraph_>0_cnt,paragraph_>50_cnt,paragraph_>75_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,...,word_12_cnt,word_13_cnt,word_14_cnt,word_15_cnt,word_len_max,word_len_mean,word_len_std,word_len_q1,word_len_q2,word_len_q3
0,000d118,1,1,1,1,1,1,1,1,1,...,6,6,5,2,25,4.378819,2.538495,3.0,4.0,5.0
1,000fe60,5,5,5,5,5,5,5,4,3,...,0,0,0,0,11,4.012048,2.060968,2.0,4.0,5.0
2,001ab80,4,4,4,4,4,4,4,4,4,...,14,10,5,2,15,4.574545,2.604621,3.0,4.0,5.0


In [ ]:
for feat in feature_names:
    train_feats = (train_feats - train_feats.mean()) / train_feats.std() 


train_feats = train_feats.dropna(axis=1)
display(train_feats)

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))


/tmp/ipykernel_24/3089841050.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_feats = (train_feats - train_feats.mean()) / train_feats.std()
/tmp/ipykernel_24/3089841050.py:2: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_feats = (train_feats - train_feats.mean()) / train_feats.std()


,paragraph_<25_cnt,paragraph_<49_cnt,paragraph_>0_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,paragraph_>300_cnt,...,word_6_cnt,word_7_cnt,word_8_cnt,word_9_cnt,word_len_max,word_len_mean,word_len_q1,word_len_q2,word_len_q3,word_len_std
0,-0.229097,-0.258149,-1.197381,-1.830000,-1.998967,-2.015836,-1.962032,-1.864468,-1.625075,-1.350034,...,0.234985,0.034206,-0.037964,0.132245,2.924296,-0.511476,0.309389,-0.104108,-1.340849,0.483009
1,-0.229097,-0.258149,0.010486,0.370921,0.519037,0.635485,0.739660,0.157654,-0.291243,-0.040702,...,-0.840922,-0.859672,-0.657133,-0.722487,-0.834356,-1.687506,-3.222668,-0.104108,-1.340849,-1.242716
2,-0.229097,-0.258149,-0.291481,-0.179309,-0.110464,-0.027345,0.064237,0.157654,0.375672,0.613965,...,0.542387,0.723769,0.909000,1.307500,0.239544,0.116111,0.309389,-0.104108,-1.340849,0.721978
3,-0.229097,-0.258149,0.010486,0.370921,0.519037,-0.027345,0.064237,0.157654,0.375672,0.613965,...,1.310892,1.489950,1.382483,1.147238,0.776495,1.473119,0.309389,-0.104108,1.605019,1.311656
4,2.222201,1.432915,0.312453,-0.179309,-0.110464,-0.027345,0.064237,0.157654,0.375672,0.613965,...,0.600025,0.953623,0.763313,1.040397,-0.028931,1.024701,0.309389,-0.104108,1.605019,1.649400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,-0.229097,-0.258149,-0.593448,-0.729539,-0.739965,-1.353006,-1.286609,-1.190427,-0.958159,-1.350034,...,-1.225174,-1.217223,-0.948507,-0.882749,-0.565881,-0.296512,0.309389,-0.104108,0.132085,0.155629
17303,-0.229097,-0.258149,0.312453,0.921152,1.148538,1.298316,1.415082,1.505735,1.709504,1.268631,...,1.714357,1.770883,1.928808,1.093817,-0.565881,0.825589,0.309389,-0.104108,1.605019,0.283174
17304,-0.229097,-0.258149,-0.593448,-0.729539,-0.739965,-0.690175,-0.611186,-0.516387,-0.291243,-0.040702,...,-1.167536,-1.166145,-1.239881,-1.310115,-0.834356,-1.040107,0.309389,-0.104108,-1.340849,-1.861784
17305,-0.229097,-0.258149,-1.197381,-1.830000,-1.998967,-2.015836,-1.962032,-1.864468,-1.625075,-1.350034,...,-0.187692,0.008667,0.034879,0.078824,-0.565881,2.243075,0.309389,4.216746,1.605019,0.898242


Features Number:  96


In [ ]:
# train_feats = train_feats.drop(columns=result)

                               
display(train_feats)

# all_train_embeds_with_features = add_new_features(all_train_embeds_with_features, new_features_train)

,paragraph_<25_cnt,paragraph_<49_cnt,paragraph_>0_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,paragraph_>300_cnt,...,word_6_cnt,word_7_cnt,word_8_cnt,word_9_cnt,word_len_max,word_len_mean,word_len_q1,word_len_q2,word_len_q3,word_len_std
0,-0.229097,-0.258149,-1.197381,-1.830000,-1.998967,-2.015836,-1.962032,-1.864468,-1.625075,-1.350034,...,0.234985,0.034206,-0.037964,0.132245,2.924296,-0.511476,0.309389,-0.104108,-1.340849,0.483009
1,-0.229097,-0.258149,0.010486,0.370921,0.519037,0.635485,0.739660,0.157654,-0.291243,-0.040702,...,-0.840922,-0.859672,-0.657133,-0.722487,-0.834356,-1.687506,-3.222668,-0.104108,-1.340849,-1.242716
2,-0.229097,-0.258149,-0.291481,-0.179309,-0.110464,-0.027345,0.064237,0.157654,0.375672,0.613965,...,0.542387,0.723769,0.909000,1.307500,0.239544,0.116111,0.309389,-0.104108,-1.340849,0.721978
3,-0.229097,-0.258149,0.010486,0.370921,0.519037,-0.027345,0.064237,0.157654,0.375672,0.613965,...,1.310892,1.489950,1.382483,1.147238,0.776495,1.473119,0.309389,-0.104108,1.605019,1.311656
4,2.222201,1.432915,0.312453,-0.179309,-0.110464,-0.027345,0.064237,0.157654,0.375672,0.613965,...,0.600025,0.953623,0.763313,1.040397,-0.028931,1.024701,0.309389,-0.104108,1.605019,1.649400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,-0.229097,-0.258149,-0.593448,-0.729539,-0.739965,-1.353006,-1.286609,-1.190427,-0.958159,-1.350034,...,-1.225174,-1.217223,-0.948507,-0.882749,-0.565881,-0.296512,0.309389,-0.104108,0.132085,0.155629
17303,-0.229097,-0.258149,0.312453,0.921152,1.148538,1.298316,1.415082,1.505735,1.709504,1.268631,...,1.714357,1.770883,1.928808,1.093817,-0.565881,0.825589,0.309389,-0.104108,1.605019,0.283174
17304,-0.229097,-0.258149,-0.593448,-0.729539,-0.739965,-0.690175,-0.611186,-0.516387,-0.291243,-0.040702,...,-1.167536,-1.166145,-1.239881,-1.310115,-0.834356,-1.040107,0.309389,-0.104108,-1.340849,-1.861784
17305,-0.229097,-0.258149,-1.197381,-1.830000,-1.998967,-2.015836,-1.962032,-1.864468,-1.625075,-1.350034,...,-0.187692,0.008667,0.034879,0.078824,-0.565881,2.243075,0.309389,4.216746,1.605019,0.898242


In [ ]:
# Paragraph
tmp = Paragraph_Preprocess(testpl)
test_feats = Paragraph_Eng(tmp)
# Sentence
tmp = Sentence_Preprocess(testpl)
test_feats = Sentence_Eng(tmp)
# Word
tmp = Word_Preprocess(testpl)
test_feats = test_feats.merge(Word_Eng(tmp), on='essay_id', how='left')
display(test_feats)


/tmp/ipykernel_24/1019194983.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
/tmp/ipykernel_24/1019194983.py:5: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('paragraph').map_elements(remove_punctuation).alias('paragraph_no_pinctuation'))
/tmp/ipykernel_24/1019194983.py:6: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  tmp = tmp.with_columns(pl.col('paragraph_no_pinctuation').map_elements(count_spelling_errors).alias("paragraph_error_num"))
/tmp/ipykernel_24/1019194983.py:7: MapWithoutReturnDtypeWarning: Cal

,essay_id,sentence_>0_cnt,sentence_>15_cnt,sentence_>50_cnt,sentence_>100_cnt,sentence_>150_cnt,sentence_>200_cnt,sentence_>250_cnt,sentence_>300_cnt,sentence_<15_cnt,...,word_12_cnt,word_13_cnt,word_14_cnt,word_15_cnt,word_len_max,word_len_mean,word_len_std,word_len_q1,word_len_q2,word_len_q3
0,000d118,14,13,11,11,7,4,3,3,1,...,6,6,5,2,25,4.378819,2.538495,3.0,4.0,5.0
1,000fe60,18,17,15,8,3,1,1,0,1,...,0,0,0,0,11,4.012048,2.060968,2.0,4.0,5.0
2,001ab80,25,24,24,15,7,1,0,0,1,...,14,10,5,2,15,4.574545,2.604621,3.0,4.0,5.0


In [ ]:
for feat in test_feats.columns.tolist():
    test_feats = (test_feats - test_feats.mean()) / test_feats.std() 

test_feats = test_feats.dropna(axis=1)
display(test_feats)


# all_test_embeds_with_features = add_new_features(all_test_embeds_with_features, new_features_test)

/tmp/ipykernel_24/1862820276.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test_feats = (test_feats - test_feats.mean()) / test_feats.std()
/tmp/ipykernel_24/1862820276.py:2: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test_feats = (test_feats - test_feats.mean()) / test_feats.std()


,sentence_<50_cnt,sentence_>0_cnt,sentence_>100_cnt,sentence_>150_cnt,sentence_>15_cnt,sentence_>200_cnt,sentence_>250_cnt,sentence_>300_cnt,sentence_>50_cnt,sentence_len_first,...,word_4_cnt,word_5_cnt,word_6_cnt,word_7_cnt,word_8_cnt,word_9_cnt,word_len_max,word_len_mean,word_len_q1,word_len_std
0,0.577350,-0.898027,-0.094916,0.577350,-0.898027,1.154701,1.091089,1.154701,-0.851064,-0.792414,...,0.295389,0.235494,0.352673,0.085784,-0.138527,-0.104828,1.10940,0.199666,0.577350,0.462294
1,0.577350,-0.179605,-0.949158,-1.154701,-0.179605,-0.577350,-0.218218,-0.577350,-0.250313,-0.331158,...,-1.114420,-1.096730,-1.128553,-1.040129,-0.923514,-0.943456,-0.83205,-1.084770,-1.154701,-1.147506
2,-1.154701,1.077632,1.044074,0.577350,1.077632,-0.577350,-0.872872,-0.577350,1.101378,1.123572,...,0.819032,0.861236,0.775880,0.954345,1.062041,1.048285,-0.27735,0.885104,0.577350,0.685211


In [ ]:
train_feat_name = train_feats.columns
test_feat_name = test_feats.columns

train_feat_name = train_feat_name.tolist()
test_feat_name = test_feat_name.tolist()
result1 = [item for item in feature_names if item not in test_feat_name]
result2 = [item for item in test_feat_name if item not in feature_names]
print(result1)
print(result2)

['paragraph_<25_cnt', 'paragraph_<49_cnt', 'paragraph_>0_cnt', 'paragraph_>100_cnt', 'paragraph_>125_cnt', 'paragraph_>150_cnt', 'paragraph_>175_cnt', 'paragraph_>200_cnt', 'paragraph_>250_cnt', 'paragraph_>300_cnt', 'paragraph_>350_cnt', 'paragraph_>400_cnt', 'paragraph_>500_cnt', 'paragraph_>50_cnt', 'paragraph_>600_cnt', 'paragraph_>700_cnt', 'paragraph_>75_cnt', 'paragraph_error_num_first', 'paragraph_error_num_last', 'paragraph_error_num_max', 'paragraph_error_num_mean', 'paragraph_error_num_min', 'paragraph_error_num_q1', 'paragraph_error_num_q3', 'paragraph_error_num_sum', 'paragraph_len_first', 'paragraph_len_last', 'paragraph_len_max', 'paragraph_len_mean', 'paragraph_len_min', 'paragraph_len_q1', 'paragraph_len_q3', 'paragraph_len_sum', 'paragraph_sentence_cnt_first', 'paragraph_sentence_cnt_last', 'paragraph_sentence_cnt_max', 'paragraph_sentence_cnt_mean', 'paragraph_sentence_cnt_min', 'paragraph_sentence_cnt_q1', 'paragraph_sentence_cnt_q3', 'paragraph_sentence_cnt_sum', '

In [ ]:
train_feats = train_feats.drop(columns=result1)
display(train_feats)

,score,sentence_<50_cnt,sentence_>0_cnt,sentence_>100_cnt,sentence_>150_cnt,sentence_>15_cnt,sentence_>200_cnt,sentence_>250_cnt,sentence_>300_cnt,sentence_>50_cnt,...,word_4_cnt,word_5_cnt,word_6_cnt,word_7_cnt,word_8_cnt,word_9_cnt,word_len_max,word_len_mean,word_len_q1,word_len_std
0,0.049380,-0.137097,-0.684344,0.460420,1.224152,-0.682953,1.608393,2.446797,4.197341,-0.760039,...,0.752523,0.286282,0.234985,0.034206,-0.037964,0.132245,2.924296,-0.511476,0.309389,0.483009
1,0.049380,-0.137097,-0.205390,-0.156400,-0.181387,-0.188201,-0.231345,0.423977,-0.419432,-0.197950,...,-0.387372,-0.697259,-0.840922,-0.859672,-0.657133,-0.722487,-0.834356,-1.687506,-3.222668,-1.242716
2,1.006411,-0.779592,0.632780,1.282846,1.224152,0.677614,-0.231345,-0.587432,-0.419432,1.066748,...,1.175912,0.748248,0.542387,0.723769,0.909000,1.307500,0.239544,0.116111,0.309389,0.721978
3,1.006411,-0.458344,0.153826,0.254813,0.872767,0.182863,0.995147,0.423977,1.119493,0.364138,...,0.622249,1.001584,1.310892,1.489950,1.382483,1.147238,0.776495,1.473119,0.309389,1.311656
4,0.049380,-0.779592,-0.444867,0.254813,0.521382,-0.311889,0.381901,1.435387,1.119493,-0.197950,...,-0.072544,0.301184,0.600025,0.953623,0.763313,1.040397,-0.028931,1.024701,0.309389,1.649400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,-0.907650,-0.137097,-1.163298,-0.978826,-0.884157,-1.177705,-0.844591,-0.587432,-0.419432,-1.322127,...,-1.418705,-1.263540,-1.225174,-1.217223,-0.948507,-0.882749,-0.565881,-0.296512,0.309389,0.155629
17303,1.006411,0.184151,1.111734,1.488452,0.872767,0.801302,0.995147,1.435387,1.119493,1.207270,...,1.447316,1.403941,1.714357,1.770883,1.928808,1.093817,-0.565881,0.825589,0.309389,0.283174
17304,-0.907650,0.505398,-0.444867,-0.978826,-1.235542,-0.435577,-0.844591,-0.587432,-0.419432,-0.760039,...,-0.951891,-1.174127,-1.167536,-1.166145,-1.239881,-1.310115,-0.834356,-1.040107,0.309389,-1.861784
17305,-1.864681,-0.779592,-0.923821,-0.567613,0.169997,-0.930329,0.381901,-0.587432,-0.419432,-0.760039,...,-0.734768,-0.354510,-0.187692,0.008667,0.034879,0.078824,-0.565881,2.243075,0.309389,0.898242


In [ ]:
train_feats = train_feats.drop(columns="score")
display(train_feats)

,sentence_<50_cnt,sentence_>0_cnt,sentence_>100_cnt,sentence_>150_cnt,sentence_>15_cnt,sentence_>200_cnt,sentence_>250_cnt,sentence_>300_cnt,sentence_>50_cnt,sentence_len_first,...,word_4_cnt,word_5_cnt,word_6_cnt,word_7_cnt,word_8_cnt,word_9_cnt,word_len_max,word_len_mean,word_len_q1,word_len_std
0,-0.137097,-0.684344,0.460420,1.224152,-0.682953,1.608393,2.446797,4.197341,-0.760039,-0.874745,...,0.752523,0.286282,0.234985,0.034206,-0.037964,0.132245,2.924296,-0.511476,0.309389,0.483009
1,-0.137097,-0.205390,-0.156400,-0.181387,-0.188201,-0.231345,0.423977,-0.419432,-0.197950,-0.592722,...,-0.387372,-0.697259,-0.840922,-0.859672,-0.657133,-0.722487,-0.834356,-1.687506,-3.222668,-1.242716
2,-0.779592,0.632780,1.282846,1.224152,0.677614,-0.231345,-0.587432,-0.419432,1.066748,0.296734,...,1.175912,0.748248,0.542387,0.723769,0.909000,1.307500,0.239544,0.116111,0.309389,0.721978
3,-0.458344,0.153826,0.254813,0.872767,0.182863,0.995147,0.423977,1.119493,0.364138,3.008490,...,0.622249,1.001584,1.310892,1.489950,1.382483,1.147238,0.776495,1.473119,0.309389,1.311656
4,-0.779592,-0.444867,0.254813,0.521382,-0.311889,0.381901,1.435387,1.119493,-0.197950,-0.321547,...,-0.072544,0.301184,0.600025,0.953623,0.763313,1.040397,-0.028931,1.024701,0.309389,1.649400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17302,-0.137097,-1.163298,-0.978826,-0.884157,-1.177705,-0.844591,-0.587432,-0.419432,-1.322127,0.068946,...,-1.418705,-1.263540,-1.225174,-1.217223,-0.948507,-0.882749,-0.565881,-0.296512,0.309389,0.155629
17303,0.184151,1.111734,1.488452,0.872767,0.801302,0.995147,1.435387,1.119493,1.207270,-0.668652,...,1.447316,1.403941,1.714357,1.770883,1.928808,1.093817,-0.565881,0.825589,0.309389,0.283174
17304,0.505398,-0.444867,-0.978826,-1.235542,-0.435577,-0.844591,-0.587432,-0.419432,-0.760039,0.014711,...,-0.951891,-1.174127,-1.167536,-1.166145,-1.239881,-1.310115,-0.834356,-1.040107,0.309389,-1.861784
17305,-0.779592,-0.923821,-0.567613,0.169997,-0.930329,0.381901,-0.587432,-0.419432,-0.760039,1.153648,...,-0.734768,-0.354510,-0.187692,0.008667,0.034879,0.078824,-0.565881,2.243075,0.309389,0.898242


In [ ]:
test_feats = test_feats.drop(columns=result2)
display(test_feats)

,sentence_<50_cnt,sentence_>0_cnt,sentence_>100_cnt,sentence_>150_cnt,sentence_>15_cnt,sentence_>200_cnt,sentence_>250_cnt,sentence_>300_cnt,sentence_>50_cnt,sentence_len_first,...,word_4_cnt,word_5_cnt,word_6_cnt,word_7_cnt,word_8_cnt,word_9_cnt,word_len_max,word_len_mean,word_len_q1,word_len_std
0,0.577350,-0.898027,-0.094916,0.577350,-0.898027,1.154701,1.091089,1.154701,-0.851064,-0.792414,...,0.295389,0.235494,0.352673,0.085784,-0.138527,-0.104828,1.10940,0.199666,0.577350,0.462294
1,0.577350,-0.179605,-0.949158,-1.154701,-0.179605,-0.577350,-0.218218,-0.577350,-0.250313,-0.331158,...,-1.114420,-1.096730,-1.128553,-1.040129,-0.923514,-0.943456,-0.83205,-1.084770,-1.154701,-1.147506
2,-1.154701,1.077632,1.044074,0.577350,1.077632,-0.577350,-0.872872,-0.577350,1.101378,1.123572,...,0.819032,0.861236,0.775880,0.954345,1.062041,1.048285,-0.27735,0.885104,0.577350,0.685211


In [ ]:
train_feat_name = train_feats.columns.tolist()
test_feat_name = test_feats.columns.tolist()

In [ ]:
print("Train: ", all_train_embeds_with_features.shape)
print("Test: ", all_test_embeds_with_features.shape)

Train:  (17307, 5382)
Test:  (3, 5382)


In [ ]:
new_features_train = train_feats[train_feat_name].values
new_features_test = test_feats[test_feat_name].values

print(new_features_train.shape)
print(new_features_test.shape)

all_train_embeds_with_features = add_new_features(all_train_embeds_with_features, new_features_train)
all_test_embeds_with_features = add_new_features(all_test_embeds_with_features, new_features_test)

print("Train: ", all_train_embeds_with_features.shape)
print("Test: ", all_test_embeds_with_features.shape)


(17307, 40)
(3, 40)
Train:  (17307, 5422)
Test:  (3, 5422)
